In [1]:
import pandas as pd
import numpy as np
import datetime
import pickle
import math

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 700)

This notebook back-calculates the PMI from the ADD (threshold = 0 degrees C).

In [63]:
# import hourly weather data
hourly_weather_df = pd.read_pickle('./../../temp_humidity_data/data/LCD/lcd_hourly.pkl')
hourly_weather_df['date'] = pd.to_datetime(hourly_weather_df['date_time'], format='%Y-%m-%d', errors='coerce')
display(hourly_weather_df.head())
display(hourly_weather_df.info())

,date_time,HourlyDryBulbTemperature,HourlyRelativeHumidity,date
0,2011-01-01 00:00:00,12.033333,90.333333,2011-01-01 00:00:00
1,2011-01-01 01:00:00,10.575000,92.250000,2011-01-01 01:00:00
2,2011-01-01 02:00:00,10.600000,92.000000,2011-01-01 02:00:00
3,2011-01-01 03:00:00,10.425000,92.000000,2011-01-01 03:00:00
4,2011-01-01 04:00:00,10.340000,92.800000,2011-01-01 04:00:00


<class 'pandas.core.frame.DataFrame'>
Int64Index: 107340 entries, 0 to 107339
Data columns (total 4 columns):
 #   Column                    Non-Null Count   Dtype         
---  ------                    --------------   -----         
 0   date_time                 107340 non-null  datetime64[ns]
 1   HourlyDryBulbTemperature  107340 non-null  float64       
 2   HourlyRelativeHumidity    107340 non-null  float64       
 3   date                      107340 non-null  datetime64[ns]
dtypes: datetime64[ns](2), float64(2)
memory usage: 4.1 MB


None

In [82]:
# convert to daily avg
avg_weather_df = hourly_weather_df.groupby([hourly_weather_df['date_time'].dt.date]).mean(numeric_only=True).reset_index()
avg_weather_df.rename(columns={"date_time": "date"}, inplace=True)
avg_weather_df['date'] = pd.to_datetime(avg_weather_df['date'], format='%Y-%m-%d')
display(avg_weather_df.head())
display(avg_weather_df.info())

,date,HourlyDryBulbTemperature,HourlyRelativeHumidity
0,2011-01-01,11.414861,91.304861
1,2011-01-02,3.506250,62.770833
2,2011-01-03,-0.079167,58.041667
3,2011-01-04,3.106250,60.166667
4,2011-01-05,0.542857,81.028075


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4473 entries, 0 to 4472
Data columns (total 3 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   date                      4473 non-null   datetime64[ns]
 1   HourlyDryBulbTemperature  4473 non-null   float64       
 2   HourlyRelativeHumidity    4473 non-null   float64       
dtypes: datetime64[ns](1), float64(2)
memory usage: 105.0 KB


None

In [184]:
# import data with ADD
data = pd.read_csv('../data/master_dataset_w_ADD.csv').sample(2000)
display(data.head())
data.info()

<ipython-input-184-26ac654e7f59>:2: DtypeWarning: Columns (10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/master_dataset_w_ADD.csv').sample(2000)


,new_id,old_id,img_path,img,correct_img_date,date_placed_ARF,PMI_days,year,sex,ancestry,est_stature_cm,est_weight_lb,est_stature_in,age_at_death,true_BP,pred_BP,pred_BP_conf,true_SOD_G,BP_of_true_SOD_G,pred_SOD_G,pred_SOD_G_conf,true_SOD_M,BP_of_true_SOD_M,donor_date,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30
460779,52a,UT52-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/52a/52a00829.46.JPG,52a00829.46.JPG,2016-08-29,2016-07-06,54.0,2016.0,Male,white,NaN,106.0,72.000000,67.0,NaN,arm,98.35,NaN,NaN,NaN,NaN,NaN,NaN,52a00829,1478.192832,1478.192832,1478.192832,1478.192832,1478.192832,1334.565441,30.05625
620913,895,UT23-13D,/da1_data/icputrd/arf/mean.js/public/anau_img3/895/89500518.33.JPG,89500518.33.JPG,2013-05-18,2013-04-15,33.0,2013.0,Male,NaN,167.64,275.0,66.000036,42.0,NaN,hand,62.18,NaN,NaN,NaN,NaN,NaN,NaN,89500518,562.023735,562.023735,552.488318,381.718750,129.561389,0.000000,0.00000
939527,d59,UT63-13D,/da1_data/icputrd/arf/mean.js/public/anau_img3/d59/d5901013.47.JPG,d5901013.47.JPG,2013-10-13,2013-10-04,9.0,2013.0,Female,NaN,154.94,190.0,61.000033,65.0,NaN,head,99.02,NaN,NaN,NaN,NaN,NaN,NaN,d5901013,186.723185,186.723185,186.723185,186.723185,66.914444,0.000000,0.00000
903183,c98,UT69-12D,/da1_data/icputrd/arf/mean.js/public/anau_img3/c98/c9801026.19.JPG,c9801026.19.JPG,2012-10-26,2012-09-13,43.0,2012.0,Male,NaN,177.80,268.0,70.000038,49.0,NaN,legs,98.64,NaN,NaN,NaN,NaN,NaN,NaN,c9801026,753.627937,753.627937,744.461270,616.472877,234.273472,0.000000,0.00000
528968,6ce,UT12-12D,/da1_data/icputrd/arf/mean.js/public/anau_img3/6ce/6ce00328.19.JPG,6ce00328.19.JPG,2012-03-28,2012-02-17,40.0,2012.0,Female,NaN,160.02,234.0,63.000034,51.0,NaN,hand,98.94,NaN,NaN,NaN,NaN,NaN,NaN,6ce00328,560.014165,551.028748,473.464512,361.307892,81.208681,0.000000,0.00000


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 460779 to 77316
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   new_id            2000 non-null   object 
 1   old_id            2000 non-null   object 
 2   img_path          2000 non-null   object 
 3   img               2000 non-null   object 
 4   correct_img_date  2000 non-null   object 
 5   date_placed_ARF   2000 non-null   object 
 6   PMI_days          2000 non-null   float64
 7   year              1994 non-null   float64
 8   sex               1994 non-null   object 
 9   ancestry          405 non-null    object 
 10  est_stature_cm    1566 non-null   float64
 11  est_weight_lb     1868 non-null   float64
 12  est_stature_in    1935 non-null   float64
 13  age_at_death      1993 non-null   float64
 14  true_BP           65 non-null     object 
 15  pred_BP           1999 non-null   object 
 16  pred_BP_conf      1999 non-null   fl

In [185]:
data['date_placed_ARF'] = pd.to_datetime(data['date_placed_ARF'], format='%Y-%m-%d')
data['correct_img_date'] = pd.to_datetime(data['correct_img_date'], format='%Y-%m-%d')
display(data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2000 entries, 460779 to 77316
Data columns (total 31 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   new_id            2000 non-null   object        
 1   old_id            2000 non-null   object        
 2   img_path          2000 non-null   object        
 3   img               2000 non-null   object        
 4   correct_img_date  2000 non-null   datetime64[ns]
 5   date_placed_ARF   2000 non-null   datetime64[ns]
 6   PMI_days          2000 non-null   float64       
 7   year              1994 non-null   float64       
 8   sex               1994 non-null   object        
 9   ancestry          405 non-null    object        
 10  est_stature_cm    1566 non-null   float64       
 11  est_weight_lb     1868 non-null   float64       
 12  est_stature_in    1935 non-null   float64       
 13  age_at_death      1993 non-null   float64       
 14  true_BP           

None

In [187]:
# derive PMI from ADD
#df_dict = data.iloc[31:32,:].to_dict('records')
df_dict = data.to_dict('records')
for row in df_dict:
    #print(row)
    discovery_date = row['correct_img_date']
    ADD = row['ADD_thres0']
    
    current_ADD = 0
    derived_PMI = -1
    current_date = discovery_date
    while (round(current_ADD, 1) < round(ADD, 1)):# rounding was needed to avoid situations like: 3.1234444 and 3.12344445
        #print(current_date)
        temp = avg_weather_df[avg_weather_df.date == current_date]['HourlyDryBulbTemperature'].values[0]
        #print(temp)
        if temp < 0:  # threshold
            current_ADD += 0
        else:
            current_ADD += temp
        
        derived_PMI += 1
        
        #print('date:'+str(start_date)+', current_ADD:'+str(current_ADD)+', temp:'+str(temp),  end="\n")
        current_date -= datetime.timedelta(days=1)
        #print(current_ADD, ADD)
    
    row['PMI_from_ADD_thres0'] = derived_PMI
    #print()

In [189]:
data2 = pd.DataFrame.from_dict(df_dict)
display(data2.head())
print(data2.shape)

,new_id,old_id,img_path,img,correct_img_date,date_placed_ARF,PMI_days,year,sex,ancestry,est_stature_cm,est_weight_lb,est_stature_in,age_at_death,true_BP,pred_BP,pred_BP_conf,true_SOD_G,BP_of_true_SOD_G,pred_SOD_G,pred_SOD_G_conf,true_SOD_M,BP_of_true_SOD_M,donor_date,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30,PMI_from_ADD_thres0
0,52a,UT52-16D,/da1_data/icputrd/arf/mean.js/public/anau_img3/52a/52a00829.46.JPG,52a00829.46.JPG,2016-08-29,2016-07-06,54.0,2016.0,Male,white,NaN,106.0,72.000000,67.0,NaN,arm,98.35,NaN,NaN,NaN,NaN,NaN,NaN,52a00829,1478.192832,1478.192832,1478.192832,1478.192832,1478.192832,1334.565441,30.05625,54
1,895,UT23-13D,/da1_data/icputrd/arf/mean.js/public/anau_img3/895/89500518.33.JPG,89500518.33.JPG,2013-05-18,2013-04-15,33.0,2013.0,Male,NaN,167.64,275.0,66.000036,42.0,NaN,hand,62.18,NaN,NaN,NaN,NaN,NaN,NaN,89500518,562.023735,562.023735,552.488318,381.718750,129.561389,0.000000,0.00000,33
2,d59,UT63-13D,/da1_data/icputrd/arf/mean.js/public/anau_img3/d59/d5901013.47.JPG,d5901013.47.JPG,2013-10-13,2013-10-04,9.0,2013.0,Female,NaN,154.94,190.0,61.000033,65.0,NaN,head,99.02,NaN,NaN,NaN,NaN,NaN,NaN,d5901013,186.723185,186.723185,186.723185,186.723185,66.914444,0.000000,0.00000,9
3,c98,UT69-12D,/da1_data/icputrd/arf/mean.js/public/anau_img3/c98/c9801026.19.JPG,c9801026.19.JPG,2012-10-26,2012-09-13,43.0,2012.0,Male,NaN,177.80,268.0,70.000038,49.0,NaN,legs,98.64,NaN,NaN,NaN,NaN,NaN,NaN,c9801026,753.627937,753.627937,744.461270,616.472877,234.273472,0.000000,0.00000,43
4,6ce,UT12-12D,/da1_data/icputrd/arf/mean.js/public/anau_img3/6ce/6ce00328.19.JPG,6ce00328.19.JPG,2012-03-28,2012-02-17,40.0,2012.0,Female,NaN,160.02,234.0,63.000034,51.0,NaN,hand,98.94,NaN,NaN,NaN,NaN,NaN,NaN,6ce00328,560.014165,551.028748,473.464512,361.307892,81.208681,0.000000,0.00000,40


(2000, 32)


In [193]:
data2[data2.PMI_days != data2.PMI_from_ADD_thres0].head()

# Note: There may be instances where data2.PMI_days != data2.PMI_from_ADD_thres0. 
# For example:
# From PMI=5 to ADD:
# Day 1: -1 degrees C
# Day 2: -1
# Day 3: 5
# Day 4: 5
# Day 5: 5
# --> ADD = 15
# From ADD = 15 -> PMI
# Day 5: 5 
# Day 4: 5
# Day 3: 5
# Day 2: -1
# Day 1: -1
# --> PMI = 3 != 5
# Usually only off by 1-2 days.

,new_id,old_id,img_path,img,correct_img_date,date_placed_ARF,PMI_days,year,sex,ancestry,est_stature_cm,est_weight_lb,est_stature_in,age_at_death,true_BP,pred_BP,pred_BP_conf,true_SOD_G,BP_of_true_SOD_G,pred_SOD_G,pred_SOD_G_conf,true_SOD_M,BP_of_true_SOD_M,donor_date,ADD_thres0,ADD_thres5,ADD_thres10,ADD_thres15,ADD_thres20,ADD_thres25,ADD_thres30,PMI_from_ADD_thres0
6,fc0,UT75-15D,/da1_data/icputrd/arf/mean.js/public/anau_img3/fc0/fc010322.13.JPG,fc010322.13.JPG,2016-03-22,2016-01-11,71.0,2015.0,Male,NaN,175.26,325.0,69.000037,70.0,NaN,backside,79.99,NaN,NaN,NaN,NaN,NaN,NaN,fc010322,512.584602,454.376407,375.482519,166.052548,40.670833,0.000000,0.0,70
7,fc0,UT75-15D,/da1_data/icputrd/arf/mean.js/public/anau_img3/fc0/fc010302.50.JPG,fc010302.50.JPG,2016-03-02,2016-01-11,51.0,2015.0,Male,NaN,175.26,325.0,69.000037,70.0,NaN,foot,67.05,NaN,NaN,NaN,NaN,NaN,NaN,fc010302,251.933234,195.451706,148.208304,15.963750,0.000000,0.000000,0.0,50
14,01e,UT20-18D,/da1_data/icputrd/arf/mean.js/public/anau_img3/01e/01e00423.16.JPG,01e00423.16.JPG,2018-04-23,2018-02-02,80.0,2018.0,Male,NaN,177.80,200.0,70.000038,72.0,NaN,head,99.96,NaN,NaN,NaN,NaN,NaN,NaN,01e00423,856.275635,814.539315,648.242986,331.822222,103.118750,0.000000,0.0,78
31,b57,UT04-15D,/da1_data/icputrd/arf/mean.js/public/anau_img3/b57/b5700723.08.JPG,b5700723.08.JPG,2015-07-23,2015-02-13,160.0,2015.0,Female,NaN,167.64,230.0,66.000036,61.0,NaN,arm,99.92,NaN,NaN,NaN,NaN,NaN,NaN,b5700723,2830.327976,2811.613393,2767.634330,2460.578656,1925.962178,832.433151,0.0,159
51,fc0,UT75-15D,/da1_data/icputrd/arf/mean.js/public/anau_img3/fc0/fc010130.04.JPG,fc010130.04.JPG,2016-01-30,2016-01-11,19.0,2015.0,Male,NaN,175.26,325.0,69.000037,70.0,NaN,torso,99.65,NaN,NaN,NaN,NaN,NaN,NaN,fc010130,42.283056,23.015625,0.000000,0.000000,0.000000,0.000000,0.0,18
